In [1]:
import re
import math

In [4]:
def getwords(doc):
    splitter = re.compile('\\W*')
    #split words by non alpha characters
    words = [s.lower() for s in splitter.split(doc) if len(s)>2 and len(s)<20 ]
    
    # return unique set of words only
    return dict([(2,1) for w in words])

In [6]:
a = getwords("hello world 12 shoildn't be teere")
print a

{2: 1}
